In [2]:
# prompt: I want to remove the 1st row from results, whereever there is NULL and not number values

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
#Entrypoint 2.x
spark = SparkSession.builder.config("spark.sql.shuffle.partitions", "2").appName("InjestionProcessing").master("local[2]").getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext




In [3]:
results = spark.read.option("header", True).parquet("/content/drive/MyDrive/CDAC PROJECT/Formula1DataAnalytics-main/Formula1DataAnalytics-main/processed/results")
races = spark.read.option("header", True).parquet("/content/drive/MyDrive/CDAC PROJECT/Formula1DataAnalytics-main/Formula1DataAnalytics-main/processed/races")
drivers = spark.read.option("header", True).parquet("/content/drive/MyDrive/CDAC PROJECT/Formula1DataAnalytics-main/Formula1DataAnalytics-main/processed/drivers")
constructors = spark.read.option("header", True).parquet("/content/drive/MyDrive/CDAC PROJECT/Formula1DataAnalytics-main/Formula1DataAnalytics-main/processed/constructors")


In [4]:
results.show(10)


+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+
|result_id|race_id|driver_id|constructor_id|number|grid|position|position_text|position_order|points|laps|       time|milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|
+---------+-------+---------+--------------+------+----+--------+-------------+--------------+------+----+-----------+------------+-----------+----+----------------+-----------------+
|     NULL|   NULL|     NULL|          NULL|  NULL|NULL|    NULL| positionText|          NULL|  NULL|NULL|       time|        NULL|       NULL|NULL|  fastestLapTime|             NULL|
|        1|     18|        1|             1|    22|   1|       1|            1|             1|  10.0|  58|1:34:50.616|     5690616|         39|   2|        1:27.452|            218.3|
|        2|     18|        2|             2|     3|   5|       2|            2| 

In [5]:
drivers.show(10)


+---------+----------+------+----+---------+----------+----------+-----------+------------------+
|driver_id|driver_ref|number|code| forename|   surname|       dob|nationality|              name|
+---------+----------+------+----+---------+----------+----------+-----------+------------------+
|        1|  hamilton|    44| HAM|    Lewis|  Hamilton|1985-01-07|    British|    Lewis Hamilton|
|        2|  heidfeld|    \N| HEI|     Nick|  Heidfeld|1977-05-10|     German|     Nick Heidfeld|
|        3|   rosberg|     6| ROS|     Nico|   Rosberg|1985-06-27|     German|      Nico Rosberg|
|        4|    alonso|    14| ALO| Fernando|    Alonso|1981-07-29|    Spanish|   Fernando Alonso|
|        5|kovalainen|    \N| KOV|   Heikki|Kovalainen|1981-10-19|    Finnish| Heikki Kovalainen|
|        6|  nakajima|    \N| NAK|   Kazuki|  Nakajima|1985-01-11|   Japanese|   Kazuki Nakajima|
|        7|  bourdais|    \N| BOU|Sébastien|  Bourdais|1979-02-28|     French|Sébastien Bourdais|
|        8| raikkone

In [6]:
constructors.show(10)


+--------------+---------------+-----------+-----------+
|constructor_id|constructor_ref|       name|nationality|
+--------------+---------------+-----------+-----------+
|          NULL| constructorRef|       name|nationality|
|             1|        mclaren|    McLaren|    British|
|             2|     bmw_sauber| BMW Sauber|     German|
|             3|       williams|   Williams|    British|
|             4|        renault|    Renault|     French|
|             5|     toro_rosso| Toro Rosso|    Italian|
|             6|        ferrari|    Ferrari|    Italian|
|             7|         toyota|     Toyota|   Japanese|
|             8|    super_aguri|Super Aguri|   Japanese|
|             9|       red_bull|   Red Bull|   Austrian|
+--------------+---------------+-----------+-----------+
only showing top 10 rows



In [7]:
constructors = constructors.filter(col(constructors.columns[0]).cast("int").isNotNull())

In [8]:
constructors.show(10)

+--------------+---------------+-----------+-----------+
|constructor_id|constructor_ref|       name|nationality|
+--------------+---------------+-----------+-----------+
|             1|        mclaren|    McLaren|    British|
|             2|     bmw_sauber| BMW Sauber|     German|
|             3|       williams|   Williams|    British|
|             4|        renault|    Renault|     French|
|             5|     toro_rosso| Toro Rosso|    Italian|
|             6|        ferrari|    Ferrari|    Italian|
|             7|         toyota|     Toyota|   Japanese|
|             8|    super_aguri|Super Aguri|   Japanese|
|             9|       red_bull|   Red Bull|   Austrian|
|            10|    force_india|Force India|     Indian|
+--------------+---------------+-----------+-----------+
only showing top 10 rows



In [9]:
results.createOrReplaceTempView("results")
races.createOrReplaceTempView("races")
drivers.createOrReplaceTempView("drivers")
constructors.createOrReplaceTempView("constructors")

In [10]:
query = """SELECT races.race_year,
constructors.name,
drivers.name,
results.position,
results.points
FROM results
JOIN drivers ON (results.driver_id = drivers.driver_id)
JOIN constructors ON (results.constructor_id = constructors.constructor_id)
JOIN races ON (results.race_id = races.race_id)
"""

In [11]:
spark.sql(query).show(10)


+---------+----------+------------------+--------+------+
|race_year|      name|              name|position|points|
+---------+----------+------------------+--------+------+
|     2008|   McLaren|    Lewis Hamilton|       1|  10.0|
|     2008|BMW Sauber|     Nick Heidfeld|       2|   8.0|
|     2008|  Williams|      Nico Rosberg|       3|   6.0|
|     2008|   Renault|   Fernando Alonso|       4|   5.0|
|     2008|   McLaren| Heikki Kovalainen|       5|   4.0|
|     2008|  Williams|   Kazuki Nakajima|       6|   3.0|
|     2008|Toro Rosso|Sébastien Bourdais|       7|   2.0|
|     2008|   Ferrari|    Kimi Räikkönen|       8|   1.0|
|     2008|BMW Sauber|     Robert Kubica|    NULL|   0.0|
|     2008|    Toyota|        Timo Glock|    NULL|   0.0|
+---------+----------+------------------+--------+------+
only showing top 10 rows



In [12]:
query = """
SELECT races.race_year,
    constructors.name AS team_name,
    drivers.name AS driver_name,
    results.position,
    results.points,
    11 - results.position AS calculated_points
FROM results
JOIN drivers ON (results.driver_id = drivers.driver_id)
JOIN constructors ON (results.constructor_id = constructors.constructor_id)
JOIN races ON (results.race_id = races.race_id)
WHERE results.position <= 10
"""

In [13]:
calculated_race_results = spark.sql(query)

In [14]:
calculated_race_results.show(10)


+---------+----------+------------------+--------+------+-----------------+
|race_year| team_name|       driver_name|position|points|calculated_points|
+---------+----------+------------------+--------+------+-----------------+
|     2008|   McLaren|    Lewis Hamilton|       1|  10.0|               10|
|     2008|BMW Sauber|     Nick Heidfeld|       2|   8.0|                9|
|     2008|  Williams|      Nico Rosberg|       3|   6.0|                8|
|     2008|   Renault|   Fernando Alonso|       4|   5.0|                7|
|     2008|   McLaren| Heikki Kovalainen|       5|   4.0|                6|
|     2008|  Williams|   Kazuki Nakajima|       6|   3.0|                5|
|     2008|Toro Rosso|Sébastien Bourdais|       7|   2.0|                4|
|     2008|   Ferrari|    Kimi Räikkönen|       8|   1.0|                3|
|     2008|   Ferrari|    Kimi Räikkönen|       1|  10.0|               10|
|     2008|BMW Sauber|     Robert Kubica|       2|   8.0|                9|
+---------+-

In [15]:
calculated_race_results.write.mode('overwrite').parquet("/content/drive/MyDrive/CDAC PROJECT/Formula1DataAnalytics-main/Formula1DataAnalytics-main/processed/calculated_race_results")

In [16]:
# prompt: print schema of calculated_race_results

calculated_race_results.printSchema()


root
 |-- race_year: integer (nullable = true)
 |-- team_name: string (nullable = true)
 |-- driver_name: string (nullable = true)
 |-- position: integer (nullable = true)
 |-- points: float (nullable = true)
 |-- calculated_points: integer (nullable = true)

